<a href="https://colab.research.google.com/github/drwitt/BME_590_Tensorflow_Deep_Learning/blob/master/Copy_of_BME_MML_Lecture_2_Tensorflow_Fundamentals_and_Python_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Dependencies

In [1]:
!pip3 install tensorflow_datasets
!pip3 install tensorflow-gpu==2.0.0-rc0

%load_ext tensorboard

!nvidia-smi

     |████████████████████████████████| 2.3MB 2.9MB/s 
     |████████████████████████████████| 380.5MB 62kB/s 
     |████████████████████████████████| 501kB 40.6MB/s 
     |████████████████████████████████| 4.3MB 29.2MB/s 
Tue Sep  3 12:46:34 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                         

# Review decorators and with blocks

Notes on with blocks: 

* useful for uploading data or files

* Notice that if anything causes an error, the with loop keeps going but will return that information on __exit__() 

* Start using with blocks with enter and exit methods

In [2]:
class controlled_execution:
    def __enter__(self):
        print('setting things up (such as opening a file)')
    def __exit__(self, type, value, traceback): # What are the __underscores__ for?
        print(value)
        print(traceback)
        print(type)
        print('removing things (such as closing a file)')

with controlled_execution():
  x = 1/0

setting things up (such as opening a file)
division by zero
<class 'ZeroDivisionError'>
removing things (such as closing a file)


ZeroDivisionError: ignored

### Decorator Notes: 

* Allows you to pass function to another "wrapper" function and use "syntactic sugar" so that you don't need to actually write def func_dec(func_inner)



In [3]:
def my_decorator(fn):
  def function_wrapper(x):
    print("Before calling " + fn.__name__)
    output = fn(x)
    print("After calling " + fn.__name__)
    return output
  return function_wrapper

@my_decorator
def my_fun(x):
  return x+x

a = my_fun(2)
print(a)

Before calling my_fun
After calling my_fun
4


My own practice:

In [17]:
def decorator_func(func_inner):
  
  def some_wrapper_func(x):
    output = func_inner(x) * 1000
    return output
  
  #Note, just return function name (not the ()) and the entire decorator func
  #returns the wrapper function within the decorator function
  return some_wrapper_func

@decorator_x
def innermost_func(var):
  return var + var

innermost_func(2)

4000

# Explore `@tf.function`
We will create the following graph in TensorFlow 2.0 using `@tf.function`

![alt text](https://miro.medium.com/max/1917/1*RfSvkiVUHHWEe805Hf1HEw.png)

In [18]:
import tensorflow as tf

@tf.function
def my_func(my_input):
    a = tf.square(my_input, name="A")
    b = tf.cos(a, name="B")
    c = tf.sin(a, name="C")   
    d = tf.add(b, c, name="D")
    e = tf.floor(b, name="E")
    f = tf.sqrt(d, name="F")
    return e,f

# We use `tf.summary` to trace the graph as it is executing.
tf.summary.trace_on(graph=True, profiler=False)
print(my_func(2.0))
with tf.summary.create_file_writer('./logs').as_default():   
    tf.summary.trace_export(
        name='tf2_graph',
        step=0,
        profiler_outdir='./logs/profiler'
    )
tf.summary.trace_off()

(<tf.Tensor: id=13, shape=(), dtype=float32, numpy=-1.0>, <tf.Tensor: id=14, shape=(), dtype=float32, numpy=nan>)


In [21]:
# We can also get an explicit tf.Graph using this with block
graph = tf.Graph()
with graph.as_default():
  my_func(2.0)
graph.as_graph_def()

node {
  name: "PartitionedCall"
  op: "PartitionedCall"
  attr {
    key: "Tin"
    value {
      list {
      }
    }
  }
  attr {
    key: "Tout"
    value {
      list {
        type: DT_FLOAT
        type: DT_FLOAT
      }
    }
  }
  attr {
    key: "_gradient_op_type"
    value {
      s: "PartitionedCall-72"
    }
  }
  attr {
    key: "config"
    value {
      s: ""
    }
  }
  attr {
    key: "config_proto"
    value {
      s: "\n\007\n\003GPU\020\001\n\007\n\003CPU\020\0012\005*\0010J\0008\001"
    }
  }
  attr {
    key: "executor_type"
    value {
      s: ""
    }
  }
  attr {
    key: "f"
    value {
      func {
        name: "__inference_my_func_71"
      }
    }
  }
}
library {
  function {
    signature {
      name: "__inference_my_func_71"
      output_arg {
        name: "identity"
        type: DT_FLOAT
      }
      output_arg {
        name: "identity_1"
        type: DT_FLOAT
      }
    }
    node_def {
      name: "A/x"
      op: "Const"
      attr {
     

In [0]:
%tensorboard --logdir='./logs'

In [23]:
# tf.Variable
# @tf.function (what happens if we try this?)
def f(const):
    a = tf.constant([[10,10],[11.,1.]])
    x = tf.constant([[1.,0.],[0.,1.]])
    b = tf.Variable(12.) + const
    y = tf.matmul(a, x) + b
    print("PRINT: ", y)
    return y

f(100)

PRINT:  tf.Tensor(
[[122. 122.]
 [123. 113.]], shape=(2, 2), dtype=float32)


<tf.Tensor: id=88, shape=(2, 2), dtype=float32, numpy=
array([[122., 122.],
       [123., 113.]], dtype=float32)>

Make a change here and compare:

In [0]:
# tf.Variable
@tf.function #(what happens if we try this?)
def f(const):
    a = tf.constant([[10,10],[11.,1.]])
    x = tf.constant([[1.,0.],[0.,1.]])
    b = tf.Variable(12.) + const
    y = tf.matmul(a, x) + b
    print("PRINT: ", y)
    return y


In [0]:
# What is the difference between class and instance?

class F:
    def __init__(self):
        self._b = None

    @tf.function
    def __call__(self, const):
        a = tf.constant([[10, 10], [11., 1.]])
        x = tf.constant([[1., 0.], [0., 1.]])
        if self._b is None:
            self._b = tf.Variable(12.)
        self._b.assign(self._b + const)
        y = tf.matmul(a, x) + self._b
        print("PRINT: ", y)
        tf.print("TF-PRINT: ", y)
        return y

f = F()
f(100.)

PRINT:  Tensor("add_1:0", shape=(2, 2), dtype=float32)
PRINT:  Tensor("add_1:0", shape=(2, 2), dtype=float32)
TF-PRINT:  [[122 122]
 [123 113]]


<tf.Tensor: id=623, shape=(2, 2), dtype=float32, numpy=
array([[122., 122.],
       [123., 113.]], dtype=float32)>